In [233]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected = True)
import plotly.graph_objs as go

print(plt.style.available) # look at available plot styles
plt.style.use('ggplot')


import os
print(os.listdir("../input"))


['grayscale', 'bmh', '_classic_test', 'classic', 'dark_background', 'Solarize_Light2', 'seaborn-dark-palette', 'seaborn-bright', 'seaborn-whitegrid', 'seaborn', 'seaborn-ticks', 'seaborn-darkgrid', 'seaborn-white', 'seaborn-notebook', 'fast', 'seaborn-deep', 'seaborn-dark', 'seaborn-colorblind', 'tableau-colorblind10', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'fivethirtyeight', 'ggplot', 'seaborn-muted', 'seaborn-talk']
['column_2C_weka.csv', 'column_3C_weka.csv']


In [247]:
data = pd.read_csv("../input/column_2C_weka.csv")
data.head(10)

,pelvic_incidence,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,63.027818,22.552586,39.609117,40.475232,98.672917,-0.254400,Abnormal
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,Abnormal
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,Abnormal
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,Abnormal
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,Abnormal
5,40.250200,13.921907,25.124950,26.328293,130.327871,2.230652,Abnormal
6,53.432928,15.864336,37.165934,37.568592,120.567523,5.988551,Abnormal
7,45.366754,10.755611,29.038349,34.611142,117.270068,-10.675871,Abnormal
8,43.790190,13.533753,42.690814,30.256437,125.002893,13.289018,Abnormal
9,36.686353,5.010884,41.948751,31.675469,84.241415,0.664437,Abnormal


In [248]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 7 columns):
pelvic_incidence            310 non-null float64
pelvic_tilt numeric         310 non-null float64
lumbar_lordosis_angle       310 non-null float64
sacral_slope                310 non-null float64
pelvic_radius               310 non-null float64
degree_spondylolisthesis    310 non-null float64
class                       310 non-null object
dtypes: float64(6), object(1)
memory usage: 17.0+ KB


In [249]:
# check for NaN values
data.isna().sum()

data.describe()

,pelvic_incidence,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis
count,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000
mean,60.496653,17.542822,51.930930,42.953831,117.920655,26.296694
std,17.236520,10.008330,18.554064,13.423102,13.317377,37.559027
min,26.147921,-6.554948,14.000000,13.366931,70.082575,-11.058179
25%,46.430294,10.667069,37.000000,33.347122,110.709196,1.603727
50%,58.691038,16.357689,49.562398,42.404912,118.268178,11.767934
75%,72.877696,22.120395,63.000000,52.695888,125.467674,41.287352
max,129.834041,49.431864,125.742385,121.429566,163.071041,418.543082


In [250]:
fig = {
    "data" : [
        {
            "x" : data["class"],
            "name" : "class",
            "marker" : {"color" : ["blue", "red"]},
            "type" : "histogram"
        }
    ],
    
    "layout" : {
        "title" : "Class Count",
        "titlefont" : {"color" : "black",
                       "size" : 20},
        "xaxis" : {"title" : "Class",
                   "color" : "black"},
        "yaxis" : {"title" : "Count",
                   "color" : "black"}
    }
}

iplot(fig)

In [251]:
class_count = data["class"].value_counts()
fig = {
    "data" : [
        {
            "labels" : class_count.index,
            "values" : class_count.values,
            "name" : "Class",
            "hoverinfo" : "label+value+name",
            "textinfo" : "percent",
            "marker" : {"line" : {"width" : 1.1, "color" : "black"}},
            "hole" : .3,
            "type" : "pie"
            
        },
    ],
    
    "layout" : {
        "title" : "Class Count",
        "titlefont" : {"size" : 20}
    }
}

iplot(fig)
plt.savefig('graph.png')

<Figure size 432x288 with 0 Axes>

In [252]:
# for KNN we need to change class names to the integer 
# Abnormal => 1  /   Normal => 0
data["class"] = [1 if each == "Abnormal" else 0 for each in data["class"]]

In [253]:
# preparing the data for classification
y = data["class"]
x_data = data.drop(["class"], axis=1)

## Normalization

In [254]:
# now we need to do normalization
x = (x_data - np.min(x_data)) / (np.max(x_data) - np.min(x_data))

## Train-Test-Split

In [255]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state = 42)

In [256]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score
knn = KNeighborsClassifier(n_neighbors = 3)

knn.fit(x_train, y_train)
y_head = knn.predict(x_test)

print(knn.score(x_test, y_test))

0.7741935483870968


In [257]:
train_accuracy = []
test_accuracy = []

for k, i in enumerate(np.arange(1, 25),1):
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(x_train, y_train)
    
    train_accuracy.append(knn.score(x_train, y_train))
    test_accuracy.append(knn.score(x_test, y_test))
print("With KNN (K=3) accuracy is: ",knn.score(x_test,y_test))
    

#plt.subplots(figsize=(18,10))
#plt.plot(np.arange(1, 25), train_accuracy, label = "Train")
#plt.plot(np.arange(1, 25), test_accuracy, label = "Test")
#plt.xlabel("Number of Neighbors (K)")
#plt.ylabel("Accuracy")
#plt.title("K vs Accuracy")
#plt.legend()
#plt.show()


With KNN (K=3) accuracy is:  0.7526881720430108


In [258]:
fig = {
    "data" : [
        {
            "x" : np.arange(1, 25),
            "y" : train_accuracy,
            "name" : "Train",
            "text" : "train",
            "marker" : {"color" : "red"},
            "type" : "scatter",
            "mode" : "lines+markers"
        },
        {
            "x" : np.arange(1, 25),
            "y" : test_accuracy,
            "name" : "Test",
            "text" : "test",
            "marker" : {"color" : "green"},
            "type" : "scatter",
            "mode" : "lines+markers"
        }
    ],
    
    "layout" : {
        "title" : "K Value vs Accuracy",
        "titlefont" : {"color" : "black",
                       "size" : 20},
        "xaxis" : {"title" : "Number of Neighbors (K)",
                   "titlefont" : {"size" : 13,
                                  "color" : "blue"}},
        "yaxis" : {"title" : "Accuracy",
                   "titlefont" : {"size" : 13,
                                  "color" : "blue"}},
        "showlegend" : True
    }
}

iplot(fig)

